In [8]:
import os
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import json

path = os.getcwd()
print(path)

/Users/anand/Documents/sayansi/workspaces/condaws/dailymed_sracping/notebook


In [9]:
'''
xml file load into memory and return BeautifulSoup Object
'''
class XmlUtil:
    def __init__(self, file_location=None,file_name=None,bs_xml=None):
        
        self.file_location = os.getcwd() if file_location is None  else file_location
        self.file_name = file_name
        self.bs_xml = None
        self.bs_xml = bs_xml if bs_xml is not None else self.get_xml_parsed(file_location,file_name)
        
    def parse_xml_file(self,file_location=None,file_name=None):
        print("parse_xml_file called")
        if file_location is None:
            file_location = self.file_location
        if file_name is None:
            file_name = self.file_name
            
        bs_xml = BeautifulSoup(open(file_location+'/'+file_name), 'xml')
        print(type(bs_xml))
        return bs_xml
    
    def get_xml_parsed(self,file_location=None,file_name=None):
        if self.bs_xml is None:
            self.bs_xml = self.parse_xml_file(file_location,file_name)
        return self.bs_xml
    
    def find_node_by_name(self, name,bs_xml = None):
        if bs_xml is None:
            bs_xml = self.bs_xml
        if name is not None:
            found_bs = xml_parser.find(name)
            return found_bs
        else:
            return None
    
    def is_attribute_data_key(self,key):
        if len([ch for ch in ["_","__"] if ch in key])>0:
            return True
        else:
            return False
        
    def get_attribute_key(self,key):
        if key is None:
            return None
        if "_" in key:
            return key.replace("_","")
        elif "__" in key:
            return key.replace("__","")
        else:
            return key
    
    def get_node_attribute_value(self, key, bs_xml =None, is_search_node = False, node_name = None):
        if bs_xml is None:
            bs_xml = self.bs_xml
        if is_search_node == True:
            bs_xml = self.find_node_by_name(bs_xml,node_name)
        if bs_xml is not None and key is not None:
            key = self.get_attribute_key(key)
            return bs_xml[key]
        
    def get_node_text_value(self, key, bs_xml = None, is_search_node =False,node_name=None):
        if bs_xml is None:
            bs_xml = self.bs_xml
        if bs_xml is not None and key is not None:
            if is_search_node == True:
                bs_xml = self.find_node_by_name(bs_xml,node_name)
            return bs_xml.contents
        else:
            return None
        
    def get_node_tree(self, key,bs_xml = None):
        if key is not None:
            bs_xml = bs_xml if bs_xml is not None else self.bs_xml
            return bs_xml[key]
        else:
            return None
    
    def get_childrens(self, bs_xml=None):
        bs_xml = bs_xml if bs_xml is not None else self.bs_xml
        if self.hasChild(bs_xml):
            return list(bs_xml.children)
    
    def hasChild(node):
        print(type(node))
        try:
            node.children
            return True
        except:
            return False

'''
    Raw XML Iteration
'''
class RawXML:
    def __init__(self, file_location=None,file_name=None,rootTree=None):        
        self.file_location = os.getcwd() if file_location is None  else file_location
        self.file_name = file_name
        self.rootTree = rootTree
        
        self.rootTree = rootTree if rootTree is not None else self.getElementTree(file_location,file_name)
        
    
    '''
    Get Element Tree of xml
    '''
    def getElementTree(self,file_location=None,file_name=None ):
        if file_location is None:
            file_location = self.file_location
        if file_name is None:
            file_name = self.file_name
            
        tree = ET.parse(file_location+'/'+file_name)
        #self.rootTree = tree
        return tree
    
    def get_unique_element(self,tree=None):
        tree = tree if tree is not None else self.rootTree
        out =set()
        for element in tree.iter():
            #print (element.tag,type(element.attrib), element.text)
            tag = element.tag.replace('{urn:hl7-org:v3}','')
            out.add(tag)
        return out
    
    def get_element_attributes(self,tagName,tree=None):
        tree = tree if tree is not None else self.rootTree
        out =set()
        found = tree.find(tagName)
        attributes = None
        if found is not None:
            attributes = found.attrib.keys()
        return attributes
    
    def iterate_xml_node_get_attr(self,tree=None):
        out ={}
        if tree is None:
            tree = self.rootTree
        elements = self.get_unique_element(tree)
        elements_dict = {key:set() for key in elements}
        print(tree)
        for element in tree.iter():
            #print("Iterateing on :",element.tag)
            childs_element = element.find(element.tag)
            #print(childs_element)
            tag = element.tag.replace('{urn:hl7-org:v3}','')
            cur_attrs = elements_dict[tag]            
            attributes = element.attrib
            if len(attributes)>0:
                elements_dict[tag]=cur_attrs.union(set(attributes))            
            
            #if childs_element is not None:
                #self.iterate_xml_node_get_attr(childs_element)
        return elements_dict
                
        

In [10]:
xmlutil = XmlUtil(None, "car.xml")

parse_xml_file called
<class 'bs4.BeautifulSoup'>


In [11]:
rxml = RawXML(None, "car.xml")
print(rxml.rootTree)

In [12]:
ott = rxml.iterate_xml_node_get_attr()

In [13]:
def doIteration():
    xml_bs = BeautifulSoup(open('car.xml'), 'xml')
    count =0
    root_nodes = set()
    chlds_lst ={};
    for children in xml_bs.find("document").children:
        count = count+1
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'+str(count))
        #chlds_lst[children.name]=children.attr
        chlds_set = node_iteration(children,children.name)
        
        root_nodes.add(children.name)
        
        chlds_lst[children.name]=chlds_set
        print('````````````````````````````````'+str(count))
    print(count) 
    return {'root':root_nodes,'chld1':chlds_lst}

def node_iteration(node, nodeName):
    child1 = set()
    if node is not None and nodeName is not None:
        #print(node.name)
        #print(node.attrs)
        soup = BeautifulSoup(str(node), 'html.parser')
        print('##########')
        #print(soup.find(nodeName))
        if soup.find(nodeName)  is not None:
            for nextChildren in soup.find(nodeName).children:
                print(str(type(nextChildren))+"--->")
                if nextChildren.name is not None:
                    child1.add(nextChildren.name)
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        return {nodeName:child1}
    
root_nodes = doIteration()
print(root_nodes)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1
````````````````````````````````1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
##########
!!!!!!!!!!!!!!!!!!!!!!!!!!!
````````````````````````````````2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~3
````````````````````````````````3
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~4
##########
!!!!!!!!!!!!!!!!!!!!!!!!!!!
````````````````````````````````4
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~5
````````````````````````````````5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~6
##########
<class 'bs4.element.NavigableString'>--->
<class 'bs4.element.Tag'>--->
<class 'bs4.element.NavigableString'>--->
<class 'bs4.element.Tag'>--->
<class 'bs4.element.NavigableString'>--->
!!!!!!!!!!!!!!!!!!!!!!!!!!!
````````````````````````````````6
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~7
````````````````````````````````7
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~8
##########
!!!!!!!!!!!!!!!!!!!!!!!!!!!
````````````````````````````````8
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~9
````````````````````````````````9
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~10
##########


In [14]:
print(xml_bs.find("document").children)

NameError: name 'xml_bs' is not defined

In [15]:
from bs4 import BeautifulSoup, NavigableString

html = '''
<html>
<body>
<div id="content">
    <h1>Welcome to Beautiful Soup</h1>
    <p>This is an example paragraph.</p>
</div>
</body>
</html>
'''

soup = BeautifulSoup(html, 'html.parser')
p_tag = soup.find('p')

# Access the parent element of the NavigableString
parent_tag = p_tag.parent

# Access the children elements of the parent element
children = parent_tag.children

# Iterate over the children elements
for child in children:
    print(child)



<h1>Welcome to Beautiful Soup</h1>


<p>This is an example paragraph.</p>


